# General Notes

When we refer to "trees" we mean equidistant metric trees, unless otherwise stated.

In [1]:
import operator

## normalize_heights

### Description

Given a list of metric trees *L* and a number *h*, normalize_heights(L,h) returns the list of trees, with all their heights ajusted to be *h*.

Specifically, the method finds the current height of the tree and subtracts the difference between the current height and h from each dictionary entry.

### Variables

*L* is a list of trees, given as a dictionary metrics on their leaves

*h* is a number, the height to normalize the trees to



In [1]:
def normalize_heights(L, h):
    for t in L:
        tree_height = max([t[k] for k in t.keys()])
        adjust = tree_height - h
        for k in t.keys():
            t[k] = t[k] - adjust
    
    return L

## get_tropical_line

### Description

Given two trees *u*, *v*, get_tropical_line(u,v) returns a list of trees (encoded as dictionary metrics on their leaves) which are (in sequence) the possible turning points on the tropical line from *u* to *v* in the space of ultrametrics. There may be repeats.

### Variables

*u, v* are trees given as a dictionary metric on their leaves.

In [4]:
def get_tropical_line(u, v, recMu=False):
    try:
        mu = sorted([u[k] - v[k] for k in u.keys()])
        L = [u]
        for scalar in mu:
            if recMu:
                L.append(({k : max(scalar + v[k], u[k]) for k in u.keys()}, scalar))
            else:
                L.append({k : max(scalar + v[k], u[k]) for k in u.keys()})
        L.append(v)
        return L
    
    except KeyError:
        print('Error: u, v must be trees on the same leaves.')

## argmaxm

### Description

Returns a list of lists encoding the argmax function on *u*. The argmax function takes an integer *i* and returns the set of pairs of leaves that achieve the *i*th largest distance in the metric. Thus, the *i*th entry of the returned list is argmax{u}{i}.

### Variables

*u* is a metric dictionary tree on its leaves.

In [2]:
def argmaxm(u):
    values = sorted(list(set(u.values())))
    return [[k for (k,v) in u.items() if v == values[-i]] for i in range(1,len(values)+1)]

## compare_trees

### Description

Determines if two trees *u*, *v* have the same tree topology. If they have the same tree topology, return True, otherwise return False.

### Variables

*u,v* are trees given as metric dictionaries on their leaves.

In [ ]:
def compare_trees(u, v):
    u_arg = argmaxm(u)
    v_arg = argmaxm(v)

    for i in range(len(u_arg)):
        try:
            if u_arg[i] != v_arg[i]:
                return False
        except KeyError:
            return False

## reduce_line

### Description

### Variables

In [3]:
def reduce_line(L):
    line = L.copy
    i = 0
    while i < len(line) - 1:
        if type(line[0]) == tuple:
            ##get to this case if we are recording other information for each tree in the line 
            if compare_trees(line[i][0], line[i+1][0]):
                ##the trees are the same, so remove the first one
                line.pop(i)
                ##stay on the same index
                i = i - 1
        ##get to this case if we are just recording the trees in the line, no other info
        elif compare_trees(line[i], line[i+1]):
            ##the trees are the same, so remove the first one
                line.pop(i)
                ##stay on the same index
                i = i - 1
        i = i + 1

## get_line_int

### Description

### Variables

In [5]:
def get_line_int(u,v):
    ##get the reduced tropical line from u to v
    line = reduce_line(get_tropical_line(u,v,True))
    
    mu = [line[0][1]]
    for i in range(1,len(line)):
        mu.append((line[i][1] + line[i-1][1])/2)
        mu.append(line[i][1])
    
    return [{k : max(m + u[k], v[k]) for k in u.keys()} for m in mu]

## left_right_split

### Description

Now I actually have no idea how this works... :(

### Variables

*a* defines the subbranch being split

In [6]:
def left_right_split(leaves, a):
    left = [leaves[0]]
    right = []
    
    i = leaves[0]
    for k in range(1, len(leaves)):
        if (i,j) in a:
            right.append(j)
        else:
            left.append(j)
    
    return left, right, len(right) == 0

## tree_rest

### Description

### Parameters

### Returns

In [7]:
def tree_rest(u, leaves):
    return {(i,j) : u[(i,j)] for (i,j) in u.keys() if i in leaves and j in leaves}

## rec_tree_paren

### Description

### Parameters

### Returns

In [8]:
def rec_tree_paren(u, leaves):
    ##if there's just two leaves left, they're as close as they can get
    if len(leaves) < 3:
        return leaves
    
    ##split at the highest branches of the current (sub)tree
    branches = split(leaves, argmaxm(u)[0])
    
    ##regard each branch as its own tree
    rests = [tree_rest(u, b) for b in branches]
    
    return [rec_tree_paren(rests[i], branches[i]) for i in range(len(branches))]

## split

### Description

### Variables

In [ ]:
def split(leaves, a):
    done = False
    while done == False:
        ##left = leftmost branch, right = everything else,
        ##done = True only if right is itself a branch
        left, right, done = leftRightSplit(leaves, a)
        
        ##add the leftmost branch to the list of splits
        splits.append(left)
        
        ##then try to split the right again
        leaves = right
        
    return splits

## print_path

### Description

### Variables

In [ ]:
def print_path(u,v):
    line = reduce_line(get_tropical_line(u,v))
    for T in line:
        print(rec_tree_paren(T, list(set([i for (i,j) in T.keys()]))))

## get_fine_codim

### Description

### Parameters

### Returns

In [9]:
def get_fine_codim(u):
    l = len(argmaxm(u))
    n_leaves = max([i for (i,j) in u.keys()])
    return n_leaves - 1 - l

## get_heights

### Description

### Parameters

### Returns

In [2]:
def get_heights(u):
    return sorted(set([h for h in u.values()]))